In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df_purchase = pd.read_csv("/kaggle/input/transaction-and-purchase-behaviour/QVI_purchase_behaviour.csv")
df_purchase

In [ ]:
df_transaction = pd.read_excel("/kaggle/input/transaction-and-purchase-behaviour/QVI_transaction_data.xlsx")
df_transaction

In [ ]:
df_purchase.info()

In [ ]:
df_transaction.info()



In [ ]:
df_transaction["DATE"] = pd.to_datetime(df_transaction["DATE"], origin = '1899-12-30', unit='D')
df_transaction.info()

In [ ]:
df_transaction

Joining data

In [ ]:
df = pd.merge(df_transaction, df_purchase, on="LYLTY_CARD_NBR", how="outer")
df

In [ ]:
df["PROD_NAME"].head(10)

In [ ]:
df_two_sales = df[df["PROD_QTY"] > 100]
df_two_sales

It looks like this customer has only had the two transactions over the year and is
not an ordinary retail customer. The customer might be buying chips for commercial
purposes instead. We'll remove this loyalty card number from further analysis.



In [ ]:
df_two_sales = df[df["LYLTY_CARD_NBR"] == 226000]
df_two_sales

In [ ]:
df.drop([71456,71457])

In [ ]:
df_by_date = df.groupby('DATE')
df_by_date.count()

There's only 364 rows, meaning only 364 dates which indicates a missing date

In [ ]:
%matplotlib inline
p = df_by_date.count().plot(figsize=(40,10))

In [ ]:
pd.date_range(start = '2018-07-01', end = '2019-06-30' ).difference(df.DATE)

We can see that the data for 25th December 2018 is missing from  our data


We can see that there is an increase in purchases in December and a break in late December.We will try to look into it.


In [ ]:
df[df['DATE'].between("2018-12-01", "2018-12-31", inclusive=True)].groupby('DATE').count().plot(kind="bar", figsize=(40,20))
df[df['DATE'].between("2018-12-01", "2018-12-31", inclusive=True)].groupby('DATE').count().plot(y = "TOT_SALES", figsize=(40,20))

We can see that the increase in sales occurs in the lead-up to Christmas and that
there are zero sales on Christmas day itself. This is due to shops being closed on
Christmas day.
Now that we are satisfied that the data no longer has outliers, we can move on to
creating other features such as brand of chips or pack size from PROD_NAME. We will
start with pack size.



In [ ]:
df.head()


In [ ]:
kettle_df = df[df["PROD_NAME"]== "Kettle 135g Swt Pot Sea Salt"]
kettle_df["PACK_SIZE(g)"] = kettle_df["PROD_NAME"].apply(lambda x:x[7:10].strip())
kettle_df['PROD_NAME'] = kettle_df['PROD_NAME'].apply(lambda x: ' '.join(x[:7].split() + x[11:].split()))
kettle_df

In [ ]:
non_kettle_df = df[df["PROD_NAME"]!= "Kettle 135g Swt Pot Sea Salt"]
non_kettle_df["PACK_SIZE(g)"] = non_kettle_df["PROD_NAME"].apply(lambda x:x[-4:-1].strip())
non_kettle_df['PROD_NAME'] = non_kettle_df['PROD_NAME'].apply(lambda x: ' '.join(x[:-4].split()))
non_kettle_df

In the above steps we have split the PROD_NAME column to give us PACK_SIZE(g) and the BRAND_NAME.
Now we concatenate the two df (kettle_df and non_kettle_df)

In [ ]:
df = pd.concat([kettle_df,non_kettle_df])
df

In [ ]:
df["PACK_SIZE(g)"] = df["PACK_SIZE(g)"].astype(int)
df.hist(column = "PACK_SIZE(g)")

We are adjusting the brannd names as there are some brand names mentioned differently but refer to the same brand.

In [ ]:
df['PROD_NAME'] = df['PROD_NAME'].replace({'Infzns': 'Infuzions',
                                'Smith': 'Smiths',
                                'Red': 'RedRD',
                                'RRD': 'RedRD',
                                'Dorito': 'Doritos',
                                'WW': 'Woolworths',
                                'Snbts': 'Sunbites',
                                'Grain': 'Sunbites',
                                'GrnWves': 'Sunbites',
                                'Natural': 'NaturalCC',
                                'NCC': 'NaturalCC',
                                'Burger': 'BurgerR',
                                'French': 'FrenchF'})

df

In [ ]:
df_customer = df.groupby("PREMIUM_CUSTOMER")
df_customer.count().plot( y="TOT_SALES", kind="bar")


The above graph shows us that "Mainstream" customers contribute most to total sales followed by "Budget" customers and "Premium" customers.

In [ ]:
df_customer.count().plot( y="PROD_QTY", kind="barh")

We can see a similar pattern for product quantity against the customer types.

In [ ]:
df_lifestage = df.groupby("LIFESTAGE")
df_lifestage.count().plot(kind="barh", y="TOT_SALES")
df_lifestage.count()

**The bar graph gives us an idea of customer_segments based on lifestage. Number of customers in each segment is displayed.**

In [ ]:
df_lifestage.count().sort_values(by="TOT_SALES").plot(kind="barh", y="TOT_SALES")

In [ ]:
df_store = df.groupby("STORE_NBR")
df_store.count().sort_values(by="TOT_SALES", ascending=False).head(10).plot(kind="bar", y="TOT_SALES")



**The above graph shows 10 stores with the most total sales.**

In [ ]:
df_store.count().sort_values(by="TOT_SALES").head(10).plot(kind="bar", y="TOT_SALES")

**The sales for the first 9 stores in the graph look too small for a year of sales. The store locations could be a reason for the meagre sales.**

**Q:- What is the most selling product?**

In [ ]:
df_prod = df.groupby("PROD_NAME")
df_prod.count().sort_values(by="TOT_SALES", ascending=False).head(10).plot.bar(y="TOT_SALES",color=['#5cb85c'])
df_prod.count().sort_values(by="TOT_SALES", ascending=False).head(10)

The top 10 products in terms of sales are very close to each other.

In [ ]:
df_package_size = df.groupby("PACK_SIZE(g)")
df_package_size.count().sort_values(by="TOT_SALES", ascending=False).head().plot(kind="barh",y="TOT_SALES")
df_package_size.count().sort_values(by="TOT_SALES", ascending=True).head().plot(kind="barh",y="TOT_SALES")

**The above graphs show the top five selling and least 5 selling packing sizes.**

In [ ]:
df_lifestage_premium = df.groupby(["LIFESTAGE","PREMIUM_CUSTOMER"])
df_lifestage_premium.count().sort_values(by="TOT_SALES").plot(kind="bar", y="TOT_SALES")

Based on Lifestage and Premium Customer the group contributing the most to the Total Sales is (Older Families, Budget) followed by (Retirees, Mainstream) and (Young Singles/Couples, Mainstream)

In [ ]:
!pip install jovian --upgrade -q
import jovian
jovian.commit(project = "Customer Analytics")

**Summary**

Top Selling Brands:-
* Thins Potato Chips Hot & Spicy
* Smiths Crnkle Chip Orgnl Big Bag
* Infuzions Thai SweetChili PotatoMix	

Top Selling Packages:-
* 175g
* 150g
* 134g

Segment which has the most number of customers contributing towards the sales:-
* OLDER SINGLES/COUPLES
* RETIREES
* OLDER FAMILIES

Based on Lifestage and Premium Customer the group contributing the most to the Total Sales is (Older Families, Budget) followed by (Retirees, Mainstream) and (Young Singles/Couples, Mainstream)

